# **Notebook 4.2: Advanced Resource Allocation with VQE (Stable Version)**

**Objective:** This notebook demonstrates how to solve a complex resource allocation problem using the **Variational Quantum Eigensolver (VQE)**. This version uses a stable configuration of Qiskit packages to ensure compatibility and avoid versioning conflicts. 📡

### **1. The Telecom Problem: Constrained Multi-Resource Allocation**

We have a scenario with multiple users and resources (e.g., Frequency Bands A, B, C). Each user must be assigned **exactly one** resource. There are costs for each assignment and penalties for interference if certain users share the same resource. Our goal is to find the assignment that minimizes the total cost.

### **2. The Scaling Challenge and the NISQ Era**

Real-world telecom problems can involve thousands of variables (qubits), far beyond the capacity of today's **NISQ (Noisy Intermediate-Scale Quantum)** computers. The purpose of this notebook is to learn the **method** on a problem that is computationally intensive for a classical simulator (`8 users * 3 resources = 24 qubits`), demonstrating the path towards solving problems that are intractable for classical machines.

---

### **3. Setup: Importing Libraries (Stable Configuration)**

In [ ]:
# Core Python libraries
import numpy as np
import qiskit

# Qiskit imports - STABLE CONFIGURATION
# Use the V1 Estimator from qiskit_aer for compatibility
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# IMPORT SamplingVQE INSTEAD OF VQE
from qiskit_algorithms.minimum_eigensolvers import SamplingVQE
from qiskit_algorithms.optimizers import SPSA

from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo


# Set a seed for reproducibility
np.random.seed(42)

print(f"Running on Qiskit version: {qiskit.__version__}")

---

### **4. Formulate the Scaled-Up Problem**

We create a problem with **8 users and 3 resources**, requiring `8 * 3 = 24` qubits.

In [ ]:
# Problem parameters - SCALED UP
num_users = 5
num_resources = 4

# For a larger problem, we generate costs and penalties randomly
np.random.seed(42)
assignment_costs = np.random.randint(1, 5, size=(num_users, num_resources))
interference_penalty = 10  # Increased penalty for a more complex problem

# --- Create the Quadratic Program ---
qp = QuadraticProgram(name="Scaled-Resource-Allocation")

# 1. Define variables (we will now have 24)
for i in range(num_users):
    for j in range(num_resources):
        qp.binary_var(name=f'x_{i}_{j}')

# 2. Define the objective function to minimize
objective = {}
for i in range(num_users):
    for j in range(num_resources):
        objective[(i * num_resources + j,)] = assignment_costs[i, j]
for j in range(num_resources):
    for i1 in range(num_users):
        for i2 in range(i1 + 1, num_users):
            var1_idx = i1 * num_resources + j
            var2_idx = i2 * num_resources + j
            objective[(var1_idx, var2_idx)] = interference_penalty
qp.minimize(linear=objective.get('linear', {}), quadratic=objective.get('quadratic', {}))

# 3. Add constraints: each user gets exactly one resource
for i in range(num_users):
    constraint_vars = {f'x_{i}_{j}': 1 for j in range(num_resources)}
    qp.linear_constraint(linear=constraint_vars, sense='==', rhs=1, name=f'user_{i}_constraint')

print("Quadratic Program for Scaled Problem Created.")
print(f"Number of variables: {qp.get_num_vars()}")

---

### **5. Convert to QUBO and Pauli Operator**

VQE works by finding the minimum eigenvalue of a Hamiltonian (a Pauli operator). We first convert our problem to a QUBO and then map that QUBO to a `SparsePauliOp`, which is the required input format for the VQE algorithm.

In [ ]:
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)

hamiltonian, offset = qubo.to_ising()
print(f"QUBO converted to a SparsePauliOp with {hamiltonian.num_qubits} qubits.")

---

### **6. Configure the VQE Solver (Stable Interface)**

To resolve the compatibility issues, we will use the **V1 Estimator** from `qiskit_aer`. The `VQE` class in `qiskit_algorithms` is designed to work with the V1 primitive interface, making this the most reliable configuration.

In [ ]:
# 1. Define the Ansatz
ansatz = EfficientSU2(num_qubits=hamiltonian.num_qubits, reps=2)

# 2. Define the Classical Optimizer
optimizer = SPSA(maxiter=100)

# 3. Choose the Quantum Backend Primitive (V1 Sampler for compatibility)
sampler = AerSampler()

# 4. Instantiate the SamplingVQE algorithm
# This is the drop-in replacement for VQE that works with MinimumEigenOptimizer
vqe_solver = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=optimizer)

# 5. Create the high-level MinimumEigenOptimizer
vqe_optimizer = MinimumEigenOptimizer(min_eigen_solver=vqe_solver)
print("Solver configured successfully with SamplingVQE for stability.")

---

### **7. Solve the Problem with VQE**

**Note:** This step will take significantly longer than a 3-user example due to the larger problem size.

In [ ]:
# The optimizer solves the original quadratic program
result = vqe_optimizer.solve(qp)

print("Result from VQE:")
print(f"Optimal solution: {result.x}")
print(f"Optimal value (total cost): {result.fval}")
print(f"Solution status: {result.status.name}")

---

### **8. Interpret and Visualize the Solution**

The solution is a binary string of length 24 (`num_users` * `num_resources`). We decode this back into a human-readable assignment.

In [ ]:
def decode_solution(result):
    assignments = {}
    for i in range(num_users):
        for j in range(num_resources):
            if result.x[i * num_resources + j] == 1:
                assignments[f'User_{i}'] = f'Resource_{j}'
    return assignments

optimal_assignments = decode_solution(result)

print("--- Optimal Resource Allocation ---")
for user, resource in optimal_assignments.items():
    print(f"{user}  ->  {resource}")
print(f"\nTotal Minimized Cost: {result.fval}")

### **9. Conclusion and Telecom Impact**

In this notebook, we successfully scaled our problem to 24 variables and solved it with VQE. This provides a tangible demonstration of the computational effort required for non-trivial quantum simulations and highlights the path toward solving problems that are intractable for classical machines.

This technique is directly applicable to many real-world 6G problems like **Network Slicing**, **Dynamic Spectrum Access**, and **Cloud-RAN Resource Management**. VQE's ability to handle complex cost functions makes it a crucial tool for the highly dynamic and optimized 6G networks of the future.